In [86]:
import mxnet as mx
from mxnet import autograd,gluon,init,nd
from mxnet.gluon import loss as gloss,nn,data as gdata
import time

# 1. BN层定义
---

In [87]:
def batch_norm(X,gamma,beta,moving_mean,moving_var,eps,momentum):
    if not autograd.is_training():
        X_hat = (X--moving_mean) / nd.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2,4)
        if len(X.shape) == 2:
            mean = X.mean(axis=0)
            var = (X-mean**2).mean(axis=0)
        else:
            mean = X.mean(axis=(0,2,3),keepdims=True)
            var = ((X-mean)**2).mean(axis=(0,2,3),keepdims=True)
        X_hat = (X-mean)/nd.sqrt(var + eps)
        
        moving_mean = momentum * moving_mean + (1-momentum) * moving_mean
        moving_var = momentum * moving_var + (1-momentum) + moving_var
    
    Y = gamma * X_hat + beta
    
    return Y,moving_mean,moving_var

In [88]:
class BatchNorm(nn.Block):
    def __init__(self,num_features,num_dims,**kwargs):
        super(BatchNorm,self).__init__(**kwargs)
        
        if num_dims ==2:
            shape = (1,num_features)
        else:
            shape = (1,num_features,1,1)
        self.gamma = self.params.get('gamma',shape = shape,init=init.One())
        self.beta = self.params.get('beta',shape = shape,init=init.Zero())
        
        self.moving_mean = nd.zeros(shape)
        self.moving_var = nd.zeros(shape)
        
        if num_dims == 2:
            shape = (1,num_features)
            
            
    def forward(self,X):
        if self.moving_mean.context != X.context:
            self.moving_mean = self.moving_mean.copyto(X.context)
            self.moving_var = self.moving_var.copyto(X.context)
        
        Y,self.moving_mean,self.moving_var = batch_norm(X,self.gamma.data(),self.beta.data(),
                                                       self.moving_mean,self.moving_var,
                                                       eps=1e-5,momentum=0.9)
        return Y 
            
        

# 2. 读取数据 + 生成iter 
---

In [89]:
batch_size = 256
mnist_train = gdata.vision.FashionMNIST(train=True)
mnist_test = gdata.vision.FashionMNIST(train=False)

transformer = gdata.vision.transforms.ToTensor()
train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),batch_size=batch_size,shuffle=True)
test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),batch_size=batch_size,shuffle=False)

# 3. 定义网络
---

In [90]:
net = nn.Sequential()
net.add(nn.Conv2D(6,kernel_size=5), # 注意，输入的不是一位线性数据，而是一张图像
        BatchNorm(6,num_dims=4),# 所以这里的dims是4，（批次，channel,长宽）。注意前面的6和conv的输出通道数要对应。
       nn.Activation('sigmoid'),
       nn.MaxPool2D(pool_size=2,strides=2),
       
       nn.Conv2D(16,kernel_size=5),
       BatchNorm(16,num_dims=4),
       nn.Activation('sigmoid'),
       nn.MaxPool2D(pool_size=2,strides=2),
       
       nn.Dense(120),
       BatchNorm(120,num_dims=2), # 注意这里的num_features也是120，但是在全连层，dims=2！！只有行和列。行是批次。
       nn.Activation('sigmoid'),
       
       nn.Dense(84),
       BatchNorm(84,num_dims=2),
       nn.Activation('sigmoid'),
       
       nn.Dense(10))


In [91]:
# 看一下输出的样子！
X = nd.random.normal(shape=(1,1,28,28))
net.initialize()
for layer in net:
    X = layer(X) # 可以看到，BN层后，数据的形状是无变化的！只是规范化了数据的值。
    print(layer.name,'output shape:\t',X.shape)

conv19 output shape:	 (1, 6, 24, 24)
batchnorm35 output shape:	 (1, 6, 24, 24)
sigmoid32 output shape:	 (1, 6, 24, 24)
pool16 output shape:	 (1, 6, 12, 12)
conv20 output shape:	 (1, 16, 8, 8)
batchnorm36 output shape:	 (1, 16, 8, 8)
sigmoid33 output shape:	 (1, 16, 8, 8)
pool17 output shape:	 (1, 16, 4, 4)
dense24 output shape:	 (1, 120)
batchnorm37 output shape:	 (1, 120)
sigmoid34 output shape:	 (1, 120)
dense25 output shape:	 (1, 84)
batchnorm38 output shape:	 (1, 84)
sigmoid35 output shape:	 (1, 84)
dense26 output shape:	 (1, 10)


# 4. 准确率
---

In [92]:
def evaluate_accuracy(data_iter,net):
    acc_sum,n = nd.array([0]),0
    for X,y in data_iter:
        y_hat = net(X)
        acc_sum += (y_hat.argmax(axis=1) == y).sum()
        n+=y.size
    return acc_sum.asscalar() / n

# 5. 训练
---

In [93]:
def train(net,train_iter,test_iter,
         batch_size,trainer,num_epochs):
    loss = gloss.SoftmaxCrossEntropyLoss()
    
    for epoch in range(num_epochs):
        # 别忘了这一行~要计算的是每一轮epoch所用的时间。
        train_l_sum,train_acc_sum,n,start = 0.0,0.0,0,time.time()
        for X,y in train_iter:
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat,y).sum()
            l.backward()
            trainer.step(batch_size)
            
            # 每一轮训练（单位是batch）都要统计这个。
            train_l_sum = l.asscalar()
            train_acc_sum = (y_hat.argmax(axis=1)==y.astype('float32')).sum().asscalar()
            n+=y.size
            
        # 但是test_acc是走完一轮epoch之后才测试的！
        # 也就是，每将数据集全部训练完后才测试net的精准度。
        test_acc = evaluate_accuracy(test_iter,net)
        print('train_l_sum:',train_l_sum / n)
        print('train_acc_sum:',train_acc_sum / n)
        print('test_acc:',test_acc)
        print('time consume:',time.time()-start)

In [94]:
lr,num_epochs,batch_size = 1.0,5,256
net.initialize(init = init.Xavier())
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train(net,train_iter,test_iter,
     batch_size,trainer,num_epochs)

D:\Anaconda3\envs\mxnet\lib\site-packages\mxnet\gluon\parameter.py:893: UserWarning: Parameter 'conv19_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
D:\Anaconda3\envs\mxnet\lib\site-packages\mxnet\gluon\parameter.py:893: UserWarning: Parameter 'conv19_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
D:\Anaconda3\envs\mxnet\lib\site-packages\mxnet\gluon\parameter.py:893: UserWarning: Parameter 'batchnorm35_gamma' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
D:\Anaconda3\envs\mxnet\lib\site-packages\mxnet\gluon\parameter.py:893: UserWarning: Parameter 'batchnorm35_beta' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
D:\Anaconda3\envs\m

MXNetError: Traceback (most recent call last):
  File "c:\jenkins\workspace\mxnet-tag\mxnet\src\io\../operator/elemwise_op_common.h", line 135
MXNetError: Check failed: assign(&dattr, vec.at(i)): Incompatible attr in node  at 1-th input: expected float32, got int32